## import library

In [187]:
import pandas as pd
import exchange_calendars as ecals
import matplotlib.pyplot as plt
import datetime
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
import os

## read data and split feature

In [214]:
data = pd.read_csv('../pre_data.csv')

In [201]:
group = ['base_hour','base_date','lane_count', 'road_rating', 'road_type', 'road_name', 'start_turn_restricted', 
         'end_turn_restricted', 'weight_restricted', 'maximum_speed_limit','target']
data = data[group]

In [215]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4131754 entries, 0 to 4131753
Data columns (total 15 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   base_hour              int64  
 1   lane_count             int64  
 2   road_rating            int64  
 3   road_type              int64  
 4   road_name              object 
 5   start_turn_restricted  object 
 6   end_turn_restricted    object 
 7   weight_restricted      float64
 8   maximum_speed_limit    float64
 9   target                 float64
 10  holiday                int64  
 11  month                  int64  
 12  year                   int64  
 13  road_visit_count       int64  
 14  visitor                int64  
dtypes: float64(3), int64(9), object(3)
memory usage: 472.8+ MB


## data remove "-"

In [203]:
data = data[data['road_name'] != "-"]

## holiday add

In [204]:
krx = ecals.get_calendar("XKRX")
df = pd.DataFrame(krx.schedule.loc["2021-01-01":"2022-12-31"])

In [205]:
open_date = pd.to_datetime(df['open'])
open_date = open_date.dt.strftime('%Y%m%d')
open_date = open_date.astype(int)
open_date = list(open_date)

In [206]:
data['holiday'] = data['base_date'].isin(open_date)

In [207]:
holiday_exist = {True : 0, False : 1}
data['holiday'] = data['holiday'].map(holiday_exist)
data['holiday'].value_counts()

0    2728064
1    1403690
Name: holiday, dtype: int64

In [208]:
data['month'] = data['base_date'].apply(lambda e: str(e)[4:6])
data['month'] = data['month'].astype(int)
data['year'] = data['base_date'].apply(lambda e: str(e)[0:4])
data['year'] = data['year'].astype(int)
del data['base_date']

In [ ]:
data.hist(bins=50, figsize=(20, 15))
plt.show()

## road count

In [209]:
road_visit_count = data['road_name'].value_counts()
tmp = pd.DataFrame({'road_name': list(road_visit_count.index), "road_visit_count": list(road_visit_count.values)})
data = data.merge(tmp, on='road_name')

## month visitor

In [210]:
additional_path = "../additional data/"
visitor_data_list = os.listdir(additional_path)
res_visitor_data_df = pd.DataFrame(columns=['year', 'month', 'visitor'])

for visitor_data in visitor_data_list:
    visitor_data_tmp = pd.read_excel(additional_path + visitor_data)[['Unnamed: 6']].iloc[2,:]
    res_visitor_data_df = res_visitor_data_df.append({'year': visitor_data[0:4], 
                                                      "month" : visitor_data[5:7], 
                                                      "visitor" : visitor_data_tmp.values[0]},
                                                    ignore_index=True)

res_visitor_data_df['month'] = res_visitor_data_df['month'].astype(int)
res_visitor_data_df['year'] = res_visitor_data_df['year'].astype(int)
data = data.merge(res_visitor_data_df, on=['year','month'])
data.to_csv("../pre_data.csv",index=False)

/tmp/ipykernel_9502/2214362831.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_visitor_data_df = res_visitor_data_df.append({'year': visitor_data[0:4],
/tmp/ipykernel_9502/2214362831.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_visitor_data_df = res_visitor_data_df.append({'year': visitor_data[0:4],
/tmp/ipykernel_9502/2214362831.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_visitor_data_df = res_visitor_data_df.append({'year': visitor_data[0:4],
/tmp/ipykernel_9502/2214362831.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_visitor_data_df = res_visitor_data_df.append({'year': visitor_data[0:4],
/tmp

## day_of_week number mapping

In [ ]:
date = {'일' : 0, '월' : 1, '화' : 2, '수' : 3, '목' : 4, '금': 5, '토': 6}
data['day_of_week'] = data['day_of_week'].map(date)

In [ ]:
data['day_of_week']

## sunday data dist

In [ ]:
sun_data = data[data['day_of_week'] == 0]
sun_data.hist(bins=50, figsize=(20, 15))
plt.show()

## tuesday data dist

In [ ]:
tue_data =  data[data['day_of_week'] == 2]
tue_data.hist(bins=50, figsize=(20, 15))
plt.show()

## zero hour dist

In [ ]:
zero_hour_data = data[data['base_hour'] == 0]
zero_hour_data.hist(bins=50, figsize=(20, 15))
plt.show()

## one hour dist

In [ ]:
one_hour_data = data[data['base_hour'] == 1]
one_hour_data.hist(bins=50, figsize=(20, 15))
plt.show()

## Feb dist

In [ ]:
feb_data = data[data['month'] == 5]
feb_data.hist(bins=50, figsize=(20, 15))
plt.show()

## nov dist

In [ ]:
nov_data = data[data['month'] == 6]
nov_data.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
data.hist(bins=50, figsize=(20, 15))

## feature importance

In [ ]:
from sklearn.feature_selection import chi2

In [ ]:
cat_list = ['lane_count', 'road_rating','maximum_speed_limit','day_of_week', 'multi_linked',
            'connect_code','road_type','holiday', 'connect_code', 'weight_restricted', 'height_restricted']
#multi_linked, day_of_week
X = data[cat_list]
y = data['target']
chi_scores = chi2(X,y)
chi_scores

In [ ]:
p_values = pd.Series(chi_scores[1],index = X.columns)
p_values.sort_values(ascending = False , inplace = True)
p_values.plot.bar()

In [ ]:
p_values

## lane_count

In [ ]:
one_lane_data = data[data['lane_count'] == 1]
one_lane_data.hist(bins=50, figsize=(20, 15))

In [ ]:
two_lane_data = data[data['lane_count'] == 2]
two_lane_data.hist(bins=50, figsize=(20, 15))

In [ ]:
three_lane_data = data[data['lane_count'] == 3]
three_lane_data.hist(bins=50, figsize=(20, 15))